In [19]:
import math
import numpy as np
import os
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pandas as pd
from io import StringIO

In [20]:
def read_gslib_formate(welllogpath="W51_por_scale_up"):
    content=""
    heads=[]
    with open(welllogpath, 'r', encoding='utf-8') as file:
        idx=0
        headnum=9999
        heads=[]
        bodys=""
        for line in file:
            if idx==1:
                headnum=int(line.strip())
            if idx>1 and idx<headnum+2:
                heads.append(line.strip())
            idx=idx+1
    with open(welllogpath, 'r', encoding='utf-8') as file:
        split_str=heads[len(heads)-1]
        content = file.read()
        contents=content.split(split_str)
        bodys=contents[1][1:]
    df = pd.read_csv(StringIO(bodys),header=None, sep=' ')
    df = df.iloc[:, :-1]
    df = df.apply(pd.to_numeric, errors='ignore')
    df.columns=heads
    return df

def pd2Gslib(S2X4_welllog_por2,df_all,petrel_formate_savepath='petrel_formate.txt'):
    df=df_all
    df1=df[['i_index unit1 scale1', 'j_index unit1 scale1','k_index unit1 scale1','x_coord unit1 scale1', 'y_coord unit1 scale1','z_coord unit1 scale1']]

    merged_df = pd.merge(df1, S2X4_welllog_por2, on=['i_index unit1 scale1', 'j_index unit1 scale1','k_index unit1 scale1','x_coord unit1 scale1', 'y_coord unit1 scale1','z_coord unit1 scale1'], how='outer')
    merged_df=merged_df.fillna(-99)
    merged_df

    merged_df.to_csv(petrel_formate_savepath, sep=' ', index=False,header=False)
    head_list=merged_df.columns.tolist()
    head_list.insert(0, str(len(merged_df.columns.tolist())))
    head_list.insert(0, "PETREL: Properties")
    head_str="\n".join(head_list)
    str_txt=head_str+"\n"
    with open(petrel_formate_savepath, 'r', encoding='utf-8') as file:
        content = file.read()
        str_txt=str_txt+content
    with open(petrel_formate_savepath, 'w', encoding='utf-8') as file:
        file.write(str_txt)
      

In [21]:
df=read_gslib_formate(welllogpath="./all_samples/W51_por_scale_up")
print(df.shape)
#S2X4的layer是18-24
S2X4_zone=df[(df['Zones(hierarchy) unit1 scale1'] > 17) & (df['Zones(hierarchy) unit1 scale1'] < 25)]
S2X4_zone[['i_index unit1 scale1', 'j_index unit1 scale1', 'k_index unit1 scale1','x_coord unit1 scale1', 'y_coord unit1 scale1', 'z_coord unit1 scale1','Zones(hierarchy) unit1 scale1']].to_csv("./all_samples/unknown_points.csv",index=False)
S2X4_welllog_por=df[(df['Zones(hierarchy) unit1 scale1'] > 17) & (df['Zones(hierarchy) unit1 scale1'] < 25)& (df['POR unit1 scale1'] >-1)]
S2X4_welllog_por['S2X4_POR unit1 scale1']=S2X4_welllog_por['POR unit1 scale1']
S2X4_welllog_por['S2X4_FACIES unit1 scale1']=S2X4_welllog_por['FACIES unit1 scale1']
S2X4_welllog_por2=S2X4_welllog_por[['i_index unit1 scale1','j_index unit1 scale1', 'k_index unit1 scale1','x_coord unit1 scale1','y_coord unit1 scale1','z_coord unit1 scale1','S2X4_FACIES unit1 scale1','S2X4_POR unit1 scale1']]
S2X4_welllog_por2.shape

(626641, 9)


C:\Users\zyytxx\AppData\Local\Temp\ipykernel_100828\3833195692.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S2X4_welllog_por['S2X4_POR unit1 scale1']=S2X4_welllog_por['POR unit1 scale1']
C:\Users\zyytxx\AppData\Local\Temp\ipykernel_100828\3833195692.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S2X4_welllog_por['S2X4_FACIES unit1 scale1']=S2X4_welllog_por['FACIES unit1 scale1']


(1612, 8)

In [22]:
pd2Gslib(S2X4_welllog_por2,df,petrel_formate_savepath='./all_samples/petrel_formate.txt')

In [27]:
S2X4_welllog_por2 = S2X4_welllog_por2.sort_values(by=['i_index unit1 scale1', 'j_index unit1 scale1', 'k_index unit1 scale1'])
grouped_df = S2X4_welllog_por2.groupby(['i_index unit1 scale1', 'j_index unit1 scale1'])
print("总共井的个数是[{}]口井！".format(str(len(grouped_df))))

train_group=[]
test_group=[]
splitsize=0.3
for idx, _group in enumerate(grouped_df):
    num_to_sample = int(len(grouped_df)/(len(grouped_df) *splitsize))
    if idx%num_to_sample==0:
        test_group.append(_group)
    else:
        train_group.append(_group)

train_group2 = [group for _, group in train_group]
train_df=pd.concat(train_group2)
train_df.to_csv("./train/known_points_train_{}.csv".format(str(1-splitsize)),index=False)
test_group2 = [group for _, group in test_group]
test_df=pd.concat(test_group2)
test_df.to_csv("./test/known_points_test_{}.csv".format(str(splitsize)),index=False)
unknown_points_grid=df[(df['Zones(hierarchy) unit1 scale1'] > 17) & (df['Zones(hierarchy) unit1 scale1'] < 25)][['i_index unit1 scale1', 'j_index unit1 scale1', 'k_index unit1 scale1', 'x_coord unit1 scale1', 'y_coord unit1 scale1', 'z_coord unit1 scale1']]
unknown_points_grid.to_csv("./all_samples/unknown_points_grid.csv",index=False)


print("总样本个数：",S2X4_welllog_por2.shape)
print("训练样本个数:",len(train_group),train_df.shape)
print("测试样本个数:",len(test_group),test_df.shape)
pd2Gslib(train_df,df,petrel_formate_savepath='./train/petrel_Gslib_formate_train_{}.txt'.format(str(1-splitsize)))
pd2Gslib(test_df,df,petrel_formate_savepath='./test/petrel_Gslib_formate_test_{}.txt'.format(str(splitsize)))


总共井的个数是[289]口井！
总样本个数： (1612, 8)
训练样本个数: 192 (1040, 8)
测试样本个数: 97 (572, 8)


In [25]:
import vtk
def pd2vtk(df):
    min_x=df["x_coord unit1 scale1"].min()
    min_y=df["y_coord unit1 scale1"].min()
    points = vtk.vtkPoints()
    scalars = vtk.vtkFloatArray()
    vtk_save_path="S2X4_welllog_por.vtk"
    for index, row in S2X4_welllog_por.iterrows():
        x=row["x_coord unit1 scale1"]-min_x
        y=row["y_coord unit1 scale1"]-min_y
        z=row["z_coord unit1 scale1"]
        v=row["POR unit1 scale1"]   
        points.InsertNextPoint(x, y, z)
        scalars.InsertNextTuple1(v)
    vtk_p = vtk.vtkPolyData()
    vtk_p.SetPoints(points)
    scalars.SetName("porosity")
    vtk_p.GetPointData().AddArray(scalars)
    vtk_p.GetPointData().SetActiveScalars("porosity")

    logWriter = vtk.vtkPolyDataWriter()
    logWriter.SetFileName(vtk_save_path)
    logWriter.SetInputData(vtk_p)
    logWriter.Write()